In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pickle 
from multiprocess import Pool
import cartopy
# Import helper functions: 
import sys
sys.path.append('../scripts/')
from load_data import load_yearly_RCM, load_ASCAT_grid

    
# Function for multiprocessing the regridding of melt data: 
def multiprocess_griddig(doy):
    '''
    Function used for multiprocessing of the regridding of data. Depending on the resolution of the RCM, the method
    of the regridding might need to be changed. 
    
    Input: doy: day of the year 
    
    Outout: grid_var: Data on the new grid for the given day of the year. 
    '''
    
    data_var = melt_data[doy]
    grid_var = np.flipud(grid_data(polar_lat, polar_lon, data_var, tif_lat, tif_lon, 'nearest'))
    return grid_var  

def perform_regridding(st_year, end_year, RCM_name, home_dir): 
    '''
    Function for permoforming the regridding of the RCM data and save annual data in a pickle file.
    
    Input:  st_year: Start year of the regridding.
            end_year: End year of the regridding.
            RCM_name: Name of the RCM model. 
            home_dir: Home directory of the project.
    
    '''
    for year in range(st_year,end_year+1): 
        print(year)
        # Now we import a year and one RCM for the full year: 
        polar_lat, polar_lon, melt_data = load_yearly_RCM(home_dir, RCM_name, year)

        # Day of the year. 
        doy = np.arange(0,np.size(melt_data,0))

        print('Begin regridding')
        # Regrid using multiprossing. Remember to check if the method is correct in the function definition.
        pool = Pool()
        pool_output = pool.map(multiprocess_griddig,doy)
        pool.close()
        pool.join 
    
        print('Regridding done')
        # Reshape results:
        data_regrid_shape = (np.size(melt_data,0), 540,299)
        melt_regridded = np.reshape(pool_output, data_regrid_shape)

        # Save as pickle:
        with open(home_dir + f'/regriddedRCM/{RCM_name}_{year}_regridded_nearest.pkl','wb') as f:
            pickle.dump(melt_regridded, f)
    

# Define which RCM and perform regridding rutine: 

In [3]:
home_dir = '/Volumes/disk3s1/enveo/'
# Choose RCM and year:
RCM_name = 'MAR'

# Load ASCAT grid which the data will be regridded to:
tif_lat, tif_lon = load_ASCAT_grid(home_dir)

# Perform regridding:
perform_regridding(2001, 2018, RCM_name, home_dir)